In [22]:
import os
from collections import defaultdict
from sklearn.model_selection import train_test_split

# Path dasar
base_path_aksarawa = r'D:\Proyek Aksara\Datasets'
dataset_paths_aksarawa = {
    'aksara_jawa': os.path.join(base_path_aksarawa, 'aksarawa'),
}

# Subfolder yang relevan
relevant_subfolders_aksarawa = {
    'aksara_jawa': ['ha', 'na', 'ca', 'ra', 'ka', 'da', 'ta', 'sa', 'wa', 'la',
                    'pa', 'dha', 'ja', 'ya', 'nya', 'ma', 'ga', 'ba', 'tha', 'nga'],  
}

# Fungsi untuk memuat path citra dan labelnya
def load_image_paths_labels(base_path_aksarawa, relevant_subfolders):
    image_paths_labels = []
    for label, folder_path in base_path_aksarawa.items():
        for subfolder in relevant_subfolders[label]:
            subfolder_path = os.path.join(folder_path, subfolder)
            if os.path.exists(subfolder_path):
                for filename in os.listdir(subfolder_path):
                    if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.gif')):
                        image_path = os.path.join(subfolder_path, filename)
                        image_paths_labels.append((image_path, subfolder))
    return image_paths_labels

# Muat data
image_paths_labels = load_image_paths_labels(dataset_paths_aksarawa, relevant_subfolders_aksarawa)

# Pisahkan path dan label
image_paths, labels = zip(*image_paths_labels)

# Label numeric
label_map = {label: idx for idx, label in enumerate(relevant_subfolders_aksarawa['aksara_jawa'])}
numeric_labels = [label_map[label] for label in labels]

# Pisahkan data menjadi train dan test (70:30)
train_paths, test_paths, train_labels, test_labels = train_test_split(
    image_paths, numeric_labels, test_size=0.3, stratify=numeric_labels, random_state=42
)


In [23]:
import shutil

# Utility function to split data
def split_data(base_path_aksarawa, output_base_path_aksarawa, split_ratio=0.7):
    for label, folder_path in base_path_aksarawa.items():
        for subfolder in relevant_subfolders_aksarawa[label]:
            subfolder_path = os.path.join(folder_path, subfolder)
            if os.path.isdir(subfolder_path):
                files = [f for f in os.listdir(subfolder_path) if f.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.gif'))]
                train_files, test_files = train_test_split(files, test_size=1-split_ratio, random_state=42)

                # Create train and test directories
                train_output_dir = os.path.join(output_base_path_aksarawa, 'train', subfolder)
                test_output_dir = os.path.join(output_base_path_aksarawa, 'test', subfolder)
                os.makedirs(train_output_dir, exist_ok=True)
                os.makedirs(test_output_dir, exist_ok=True)

                for file in train_files:
                    shutil.copy(os.path.join(subfolder_path, file), os.path.join(train_output_dir, file))
                for file in test_files:
                    shutil.copy(os.path.join(subfolder_path, file), os.path.join(test_output_dir, file))

# Apply the split
output_base_path_aksarawa = r'D:\Proyek Aksara\Datasets_split_aksarawa'
split_data(dataset_paths_aksarawa, output_base_path_aksarawa)


In [24]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

# Path dataset split
train_dir = os.path.join(output_base_path_aksarawa, 'train')
test_dir = os.path.join(output_base_path_aksarawa, 'test')

# ImageDataGenerator untuk augmentasi dan preprocessing
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

test_datagen = ImageDataGenerator(rescale=1./255)

# Buat generator
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(128, 128),
    batch_size=32,
    class_mode='sparse'
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(128, 128),
    batch_size=32,
    class_mode='sparse'
)


Found 6988 images belonging to 20 classes.
Found 3007 images belonging to 20 classes.


In [31]:
classes = sorted(os.listdir(train_dir))

# Tampilkan kelas
print("Kelas yang ada dalam dataset:")
for idx, cls in enumerate(classes):
    print(f"{idx + 1}. {cls}")


Kelas yang ada dalam dataset:
1. ba
2. ca
3. da
4. dha
5. ga
6. ha
7. ja
8. ka
9. la
10. ma
11. na
12. nga
13. nya
14. pa
15. ra
16. sa
17. ta
18. tha
19. wa
20. ya


In [48]:
import os

output_base_path_aksarawa = r'D:\Proyek Aksara\Datasets_split_aksarawa'

# Mengumpulkan label dari struktur output
def get_labels_from_output(output_base_path):
    labels = {}
    for split_folder in ['train', 'test']:  # Iterasi melalui folder train dan test
        split_path = os.path.join(output_base_path, split_folder)
        for label in os.listdir(split_path):
            label_path = os.path.join(split_path, label)
            if os.path.isdir(label_path):
                labels[label] = split_folder  # Menyimpan label split (train/test) untuk setiap folder
    return labels

# Memanggil fungsi untuk mendapatkan label dari struktur output
output_labels = get_labels_from_output(output_base_path_aksarawa)

# Menampilkan label untuk setiap subfolder di dalam direktori train atau test
for label, split in output_labels.items():
    print(f"Folder '{label}' di dalam direktori '{split}' dilabeli sebagai: {label}")


Folder 'ba' di dalam direktori 'test' dilabeli sebagai: ba
Folder 'ca' di dalam direktori 'test' dilabeli sebagai: ca
Folder 'da' di dalam direktori 'test' dilabeli sebagai: da
Folder 'dha' di dalam direktori 'test' dilabeli sebagai: dha
Folder 'ga' di dalam direktori 'test' dilabeli sebagai: ga
Folder 'ha' di dalam direktori 'test' dilabeli sebagai: ha
Folder 'ja' di dalam direktori 'test' dilabeli sebagai: ja
Folder 'ka' di dalam direktori 'test' dilabeli sebagai: ka
Folder 'la' di dalam direktori 'test' dilabeli sebagai: la
Folder 'ma' di dalam direktori 'test' dilabeli sebagai: ma
Folder 'na' di dalam direktori 'test' dilabeli sebagai: na
Folder 'nga' di dalam direktori 'test' dilabeli sebagai: nga
Folder 'nya' di dalam direktori 'test' dilabeli sebagai: nya
Folder 'pa' di dalam direktori 'test' dilabeli sebagai: pa
Folder 'ra' di dalam direktori 'test' dilabeli sebagai: ra
Folder 'sa' di dalam direktori 'test' dilabeli sebagai: sa
Folder 'ta' di dalam direktori 'test' dilabeli seb

BASE MODEL

In [28]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping


# Path dataset split
train_dir = os.path.join(output_base_path_aksarawa, 'train')
test_dir = os.path.join(output_base_path_aksarawa, 'test')

# ImageDataGenerator untuk augmentasi dan preprocessing
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

test_datagen = ImageDataGenerator(rescale=1./255)

# Buat generator
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(128, 128),
    batch_size=32,
    class_mode='sparse'
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(128, 128),
    batch_size=32,
    class_mode='sparse'
)

model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(256, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(512, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(len(relevant_subfolders_aksarawa['aksara_jawa']), activation='softmax')  
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

#early stopping for avoid overfitting
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)


# Train model
steps_per_epoch = train_generator.samples // train_generator.batch_size
validation_steps = test_generator.samples // test_generator.batch_size

history = model.fit(
    train_generator,
    epochs=50,
    validation_data=test_generator,
    steps_per_epoch=steps_per_epoch,
    validation_steps=validation_steps
)

# Evaluasi model
test_loss, test_acc = model.evaluate(test_generator)
print(f'Akurasi pada data pengujian: {test_acc:.2f}')

Found 6988 images belonging to 20 classes.
Found 3007 images belonging to 20 classes.
Epoch 1/50
218/218 ━━━━━━━━━━━━━━━━━━━━ 75s 314ms/step - accuracy: 0.0598 - loss: 2.9638 - val_accuracy: 0.0689 - val_loss: 2.9348
Epoch 2/50
218/218 ━━━━━━━━━━━━━━━━━━━━ 0s 438us/step - accuracy: 0.1562 - loss: 2.8870 - val_accuracy: 0.1290 - val_loss: 2.9758
Epoch 3/50
218/218 ━━━━━━━━━━━━━━━━━━━━ 70s 317ms/step - accuracy: 0.0612 - loss: 2.9424 - val_accuracy: 0.0706 - val_loss: 2.9302
Epoch 4/50
218/218 ━━━━━━━━━━━━━━━━━━━━ 0s 425us/step - accuracy: 0.0938 - loss: 2.9205 - val_accuracy: 0.1290 - val_loss: 2.8487
Epoch 5/50
218/218 ━━━━━━━━━━━━━━━━━━━━ 75s 341ms/step - accuracy: 0.0644 - loss: 2.9380 - val_accuracy: 0.0692 - val_loss: 2.9425
Epoch 6/50
218/218 ━━━━━━━━━━━━━━━━━━━━ 0s 422us/step - accuracy: 0.0000e+00 - loss: 2.9418 - val_accuracy: 0.0968 - val_loss: 2.9619
Epoch 7/50
218/218 ━━━━━━━━━━━━━━━━━━━━ 73s 332ms/step - accuracy: 0.0691 - loss: 2.9355 - val_accuracy: 0.0729 - val_loss: 2.9

In [7]:
# Save the model
model.save('aksaraJawa_classifier_model.h5')

In [8]:
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import img_to_array, load_img
import numpy as np
from ipywidgets import FileUpload, Image, VBox, Output, Label
from IPython.display import display
import io

# Load the model
model = load_model('aksaraJawa_classifier_model.h5')

# Class labels
class_labels = ['ba','ca','da','dha','ga','ha','ja','ka','la','ma','na','nga','nya','pa','ra','sa','ta','tha','wa','ya']

# Function to classify image
def classify_image(img_array):
    img_array = np.expand_dims(img_array, axis=0) / 255.0
    predictions = model.predict(img_array)
    print(f"Raw predictions: {predictions}")  # Debug: Print raw predictions
    predicted_class_idx = np.argmax(predictions)
    print(f"Predicted class index: {predicted_class_idx}")  # Debug: Print predicted class index
    predicted_class = class_labels[predicted_class_idx]
    return predicted_class

# Widget for file upload
upload_widget = FileUpload(accept='image/*', multiple=False)
output = Output()

def on_upload_change(change):
    with output:
        output.clear_output()
        if not upload_widget.value:
            return
        
        # Get the uploaded file
        uploaded_file = upload_widget.value[0]  # Access the first item of the tuple
        print(f"Uploaded file: {uploaded_file}")  # Debug: Print uploaded file information
        content = uploaded_file['content']  # Access the content directly
        
        try:
            # Convert content to image array
            img = load_img(io.BytesIO(content), target_size=(128, 128))
            img_array = img_to_array(img)
            
            # Classify the uploaded image
            predicted_class = classify_image(img_array)
            
            # Display the uploaded image and classification result
            display_img = Image(value=content, format='png', width=300, height=300)
            result_label = Label(f'Classified as: {predicted_class}')
            display(VBox([display_img, result_label]))
        
        except Exception as e:
            print(f"Error processing image: {e}")

upload_widget.observe(on_upload_change, names='value')

# Display the upload widget and output
display(VBox([upload_widget, output]))
